In [1]:
import astropy
from astropy.io import fits
import numpy as np
import matplotlib.pyplot as plt


In [2]:
cd ~/Documents/M2_Python/PROJET

/nethome/bordier/Documents/M2_Python/PROJET


In [3]:
CMB=fits.open('COM_PowerSpect_CMB_R2.02.fits')
hdu=CMB[8]      #L=30-2508 uniquement dans le header 8
header=hdu.header
data=hdu.data

a1=data['ELL']


print header

XTENSION= 'BINTABLE'           /Written by IDL:  Tue Jun  9 11:22:40 2015       BITPIX  =                    8 /                                                NAXIS   =                    2 /Binary table                                    NAXIS1  =                   10 /Number of bytes per row                         NAXIS2  =                 2479 /Number of rows                                  PCOUNT  =                    0 /Random parameter count                          GCOUNT  =                    1 /Group count                                     TFIELDS =                    3 /Number of columns                               COMMENT                                                                         COMMENT  *** End of mandatory fields ***                                        COMMENT                                                                         EXTVER  =                    1 /Extension version                               DATE    = '2015-06-09'         /Creation

In [54]:
data=hdu.data
l=data['ELL']
Dl=data['D_ELL']
Error=data['ERR']

index1=np.where((l>=50)&(l<500))

l=l[index1]
Dl=Dl[index1]
Error=Error[index1]

#print l,len(l)

plt.figure(1)
#plt.plot(l,Dl/np.max(Dl),'b.')
#plt.plot(l,modele/np.max(modele),'r')
plt.errorbar(l,Dl,Error,ecolor='r')
plt.title('Dl en fonction de l')
plt.xlabel('l')
plt.ylabel('Dl')
plt.show()
plt.clf()


In [9]:
Dplanck=open("COM_PowerSpect_CMB-base-plikHM-TT-lowTEB-minimum-theory_R2.02.txt")
Planck=np.genfromtxt("COM_PowerSpect_CMB-base-plikHM-TT-lowTEB-minimum-theory_R2.02.txt",skip_header=1)
#print Planck
Dplanck=Dplanck.read()

#print Dplanck
#print Planck[:,0]

Planck_model=Planck[:,1]
l_Planck=Planck[:,0]
index1=np.where(l_Planck>=50)

Planck_model=Planck_model[index1]

In [23]:
def modele2(teta,l0):  #teta=(A,n,hp,lp,wp)
    exp=np.exp(-(l-teta[3])**2/(2*teta[4]**2))
    L= teta[0]*(l/l0)**teta[1]
    return L*(1+teta[2]*exp)

def jacobien2(teta,l0):   #teta=(A,n,hp,lp,wp)
    teta=np.array(teta)
    nombre_para=len(teta)
    J=np.zeros((len(Dl),nombre_para))
    for i in range (0,len(J)):
        exp=np.exp(-(l[i]-teta[3])**2/(2*teta[4]**2))
        L= teta[0]*(l[i]/l0)**teta[1]  #A*l^n/lo
        J[i,0]= (l[i]/l0)**teta[1]*(1+teta[2]*exp)   #derivee selon A
        J[i,1]=L*np.log(l[i]/l0)*(1+teta[2]*exp)  #derivee selon n
        J[i,2]=L*exp #derivee selon hp
        J[i,3]=L*teta[2]*(l[i]-teta[3])/teta[4]**2*exp   #derivee selon lp
        J[i,4]=L*2*(l[i]-teta[3])**2/teta[4]**3*teta[2]*exp   #derivee selon wp
        
    return J

In [57]:
#teta=(1.15,0.17,5000.,220.,92.)     #On trouve les paramètres en fonction de l'allure des Dl
#tetaLM1=[  1.55933375e+03,2.42232956e-01 ,2.72230717e+00 ,2.07495189e+02, 7.41392799e+01]
tetaLM0001=[  1.55957038e+03, 2.42085338e-01, 2.72183342e+00,2.07503266e+02,7.41388733e+01]
modele=modele2(tetaLM0001,200.)
plt.figure(1)
#plt.plot(l,Dl/np.max(Dl),'b.')
#plt.plot(l,modele/np.max(modele),'r')
plt.plot(l,Dl,'b.',label='Donnees')
plt.plot(l,modele,'r',label='Modele Gaussien')
plt.title('Donnees et Modele2 en fonction de l')
plt.xlabel('l')
plt.ylabel('Dl')
plt.legend()

plt.show()
plt.clf()

In [56]:
sigma=np.diag(Error)
inv_sigma=np.linalg.inv(sigma)

teta= (1.15,0.17,5000.,220.,92.) 
lbda=0.
A=[]
VRAISteta=[]
VRAISdteta=[]
for i in range(50):
    #teta[0]=A[i]
    J=jacobien2(teta,200.)
    trans_Jac=J.T
    model=modele2(teta,200.)
    M=trans_Jac.dot(inv_sigma.dot(J))
    N=trans_Jac.dot(inv_sigma.dot(Dl-model))
    dteta=np.linalg.inv(M+lbda*np.diag(M)).dot(N)
    teta=teta+dteta
    
    #teta=teta+dteta
    vrais=0.5*(Dl-model).dot(inv_sigma.dot(Dl-model))
    vrais2=0.5*(Dl-modele2(teta,200.)).dot(inv_sigma.dot(Dl-modele2(teta,200.)))
    VRAISteta.append(vrais)
    VRAISdteta.append(vrais2)
    A.append(teta[0])
    if abs(vrais2-vrais)>0.001:
        pass
    else:
        break
        
     #A=A+dteta
    
    print i, teta, dteta, vrais, vrais2

0 [  1.98697762e+03  -1.45875202e-01  -8.63055063e+06   2.26493791e+02
   8.00170107e+01] [  1.98582762e+03  -3.15875202e-01  -8.63555063e+06   6.49379123e+00
  -1.19829893e+01] 1206684.73296 4.56286625244e+19
1 [  2.39719774e+03  -1.45876042e-01   1.73064083e+06   2.26493816e+02
   8.00170116e+01] [  4.10220123e+02  -8.40411829e-07   1.03611915e+07   2.52585710e-05
   9.33046636e-07] 4.56286625244e+19 2.67051844707e+18
2 [  2.24741876e+03  -1.45872355e-01   1.08503625e+05   2.26493706e+02
   8.00170103e+01] [ -1.49778984e+02   3.68753246e-06  -1.62213721e+06  -1.10175812e-04
  -1.32655697e-06] 2.67051844707e+18 9.22614934982e+15
3 [  2.24036120e+03  -1.45809445e-01   3.41560655e+02   2.26491828e+02
   8.00169898e+01] [ -7.05755442e+00   6.29092982e-05  -1.08162064e+05  -1.87863792e-03
  -2.04527761e-05] 9.22614934982e+15 90133974121.4
4 [  2.20767106e+03  -1.26974973e-01   5.91066797e+00   2.25925425e+02
   8.00091827e+01] [ -3.26901431e+01   1.88344723e-02  -3.35649987e+02  -5.664027